In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')

In [ ]:
test=pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')

In [ ]:
train.head(5)

In [ ]:
train.shape

In [ ]:
train.isna().sum()

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
def data_overview(df, name):
    print(f"\n{name} Dataset Overview:\n")
    print("Shape:", df.shape)
    print("\nColumns and Data Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nStatistical Summary:\n", df.describe(include='all'))

In [ ]:
data_overview(train, "Train")
data_overview(test, "Test")

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [ ]:
for df in [train, test]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6])

In [ ]:
train.fillna(0, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
le_country = LabelEncoder()
train['country_encoded'] = le_country.fit_transform(train['country'])
test['country_encoded'] = le_country.transform(test['country'])

In [ ]:
le_store = LabelEncoder()
train['store_encoded'] = le_store.fit_transform(train['store'])
test['store_encoded'] = le_store.transform(test['store'])

In [ ]:
le_item = LabelEncoder()
train['item_encoded'] = le_item.fit_transform(train['product'])
test['item_encoded'] = le_item.transform(test['product'])

In [ ]:
features = ['country_encoded', 'store_encoded', 'item_encoded', 'year', 'month', 'day', 'day_of_week', 'is_weekend']
X = train[features]
y = train['num_sold']

In [ ]:
print("Missing values in training features:")
print(X.isnull().sum())


In [ ]:
X.fillna(0, inplace=True)

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
print("NaN check after scaling:")
print("X_train contains NaNs:", np.isnan(X_train).any())
print("X_valid contains NaNs:", np.isnan(X_valid).any())

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error


In [ ]:
gbr = GradientBoostingRegressor(random_state=42)

In [ ]:
gbr.fit(X_train, y_train)

In [ ]:
y_pred = gbr.predict(X_valid)
mape = mean_absolute_percentage_error(y_valid, y_pred)
print("Validation MAPE:", mape)

In [ ]:
test = test.sort_values(by=['store', 'product', 'date'])

In [ ]:
features = ['country_encoded', 'store_encoded', 'item_encoded', 'year', 'month', 'day', 'day_of_week', 'is_weekend']

In [ ]:
test_scaled = scaler.transform(test[features])

In [ ]:
test['num_sold'] = gbr.predict(test_scaled)

In [ ]:
submission = test[['id', 'num_sold']]
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")